# sentiment_classification_with_neural_network

by 陈炳炎（bjchenbingyan@126.com）
* blog：http://www.intelligentdata.cn/


###     本项目使用神经网络来对英文文本进行情感分析。本项目使用的核心算法是三层神经网络（也就是多层感知机模型：Multilayer Perceptron），包含输入层、隐藏层和输出层。并采用BP算法（误差逆传播算法）进行建模和权值更新。

In [1]:
import numpy as np
from collections import Counter
import time
import sys

## 1.读取数据

In [2]:
f = open("./data/reviews.txt", 'r')
reviews = list(map(lambda x:x[:-1], f.readlines()))
f.close()

f = open("./data/labels.txt", 'r')
labels = list(map(lambda x:x[:-1].upper(), f.readlines()))
f.close()

In [3]:
print("length of reviews: \t", len(reviews))
print("length of labels: \t", len(labels))

length of reviews: 	 25000
length of labels: 	 25000


In [4]:
print("content of the first review: \t\n", reviews[0])
print("==============================================")
print("content of the first label: \t\n", labels[0])

content of the first review: 	
 bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
content of the first label: 	
 POSITIVE


## 2. 统计词频
#### 这一节统计所有评论中出现的词汇，分别存入三个集合中：正面评论、反面评论和全部评论。通过分析这三个集合的内容来找出具有正面倾向、反面倾向和中立倾向的词汇的特点。

In [5]:
#定义三个计数器来记录评论出现的词汇
positive_count = Counter()
negative_count = Counter()
total_count = Counter()

In [6]:
#循环遍历数据集的所有单词，每出现一个单词，就向对应计数器和总计数器增加1
for i in range(len(reviews)):
    if(labels[i] == 'POSITIVE'):
        for word in reviews[i].split(" "):
            positive_count[word] += 1
            total_count[word] += 1
    if(labels[i] == 'NEGATIVE'):
        for word in reviews[i].split(" "):
            negative_count[word] += 1
            total_count[word] += 1

In [7]:
#查看三个计数器中出现最频繁的词汇，分析三个计数器中高频词汇有何异同，为后续的进一步数据处理做准备
positive_count.most_common()

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235),
 ('it', 48025),
 ('i', 40743),
 ('that', 35630),
 ('this', 35080),
 ('s', 33815),
 ('as', 26308),
 ('with', 23247),
 ('for', 22416),
 ('was', 21917),
 ('film', 20937),
 ('but', 20822),
 ('movie', 19074),
 ('his', 17227),
 ('on', 17008),
 ('you', 16681),
 ('he', 16282),
 ('are', 14807),
 ('not', 14272),
 ('t', 13720),
 ('one', 13655),
 ('have', 12587),
 ('be', 12416),
 ('by', 11997),
 ('all', 11942),
 ('who', 11464),
 ('an', 11294),
 ('at', 11234),
 ('from', 10767),
 ('her', 10474),
 ('they', 9895),
 ('has', 9186),
 ('so', 9154),
 ('like', 9038),
 ('about', 8313),
 ('very', 8305),
 ('out', 8134),
 ('there', 8057),
 ('she', 7779),
 ('what', 7737),
 ('or', 7732),
 ('good', 7720),
 ('more', 7521),
 ('when', 7456),
 ('some', 7441),
 ('if', 7285),
 ('just', 7152),
 ('can', 7001),
 ('story', 6780),
 ('time', 6515),
 ('my', 6488),
 ('g

In [8]:
negative_count.most_common()

[('', 561462),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327),
 ('i', 46880),
 ('in', 43753),
 ('this', 40920),
 ('that', 37615),
 ('s', 31546),
 ('was', 26291),
 ('movie', 24965),
 ('for', 21927),
 ('but', 21781),
 ('with', 20878),
 ('as', 20625),
 ('t', 20361),
 ('film', 19218),
 ('you', 17549),
 ('on', 17192),
 ('not', 16354),
 ('have', 15144),
 ('are', 14623),
 ('be', 14541),
 ('he', 13856),
 ('one', 13134),
 ('they', 13011),
 ('at', 12279),
 ('his', 12147),
 ('all', 12036),
 ('so', 11463),
 ('like', 11238),
 ('there', 10775),
 ('just', 10619),
 ('by', 10549),
 ('or', 10272),
 ('an', 10266),
 ('who', 9969),
 ('from', 9731),
 ('if', 9518),
 ('about', 9061),
 ('out', 8979),
 ('what', 8422),
 ('some', 8306),
 ('no', 8143),
 ('her', 7947),
 ('even', 7687),
 ('can', 7653),
 ('has', 7604),
 ('good', 7423),
 ('bad', 7401),
 ('would', 7036),
 ('up', 6970),
 ('only', 6781),
 ('more', 6730),
 ('

In [9]:
total_count.most_common()

[('', 1111930),
 ('the', 336713),
 ('.', 327192),
 ('and', 164107),
 ('a', 163009),
 ('of', 145864),
 ('to', 135720),
 ('is', 107328),
 ('br', 101872),
 ('it', 96352),
 ('in', 93968),
 ('i', 87623),
 ('this', 76000),
 ('that', 73245),
 ('s', 65361),
 ('was', 48208),
 ('as', 46933),
 ('for', 44343),
 ('with', 44125),
 ('movie', 44039),
 ('but', 42603),
 ('film', 40155),
 ('you', 34230),
 ('on', 34200),
 ('t', 34081),
 ('not', 30626),
 ('he', 30138),
 ('are', 29430),
 ('his', 29374),
 ('have', 27731),
 ('be', 26957),
 ('one', 26789),
 ('all', 23978),
 ('at', 23513),
 ('they', 22906),
 ('by', 22546),
 ('an', 21560),
 ('who', 21433),
 ('so', 20617),
 ('from', 20498),
 ('like', 20276),
 ('there', 18832),
 ('her', 18421),
 ('or', 18004),
 ('just', 17771),
 ('about', 17374),
 ('out', 17113),
 ('if', 16803),
 ('has', 16790),
 ('what', 16159),
 ('some', 15747),
 ('good', 15143),
 ('can', 14654),
 ('more', 14251),
 ('she', 14223),
 ('when', 14182),
 ('very', 14069),
 ('up', 13291),
 ('time', 127

* 从上面的三个计数器中的高频词汇，我们可以看出他们相差不大，大部分是一些停止词和中立词，这些词汇在句子对情感分析作用不大，但是出现频次又很高。
* 在进行文本分析时，停止词不仅不能提高模型的分析能力，而且引入了大量的噪声，对模型的准确性有很大的影响，因此一般在进行数据预处理阶段，需要去除停止词。
* 下面我们通过数学变换，将正面评论计数器和反面评论计数器中的相同词的词频进行相除处理，这样理论上，对于停止词相处结果会在1附近，而具有情感偏向的词会由于出现频次不同而偏离1，下面的代码对该理论进行验证。

In [35]:
pos_neg_ratios = Counter()
for word, cnt in total_count.most_common():
    if(cnt > 100):
        pos_neg_ratio = positive_count[word] / float(negative_count[word] + 1)
        pos_neg_ratios[word] = pos_neg_ratio

In [36]:
print("pos_neg_ratio for 'the' = {}".format(pos_neg_ratios['the']))
print("pos_neg_ratio for 'great' = {}".format(pos_neg_ratios['great']))
print("pos_neg_ratio for 'bad' = {}".format(pos_neg_ratios['bad']))

pos_neg_ratio for 'the' = 1.0607993145235326
pos_neg_ratio for 'great' = 2.4305187429004165
pos_neg_ratio for 'bad' = 0.2576330721426641


* 从上面的输出可以看出，词汇the的输出是1.06，great的输出的2.43，bad的输出是0.25。也就是说，the在正面评论和反面评论的出现频次差不多。而具有正面倾向的词汇grea在正向评论中出现的频次是反向评论的2.43倍。同理，bad在反向评论中出现的频次是正向的4倍左右。
* 但是，进一步分析，我们发现，对于正向词汇，计算得到的数值取值范围是[1:+∞），而反向词汇的取值范围是[0:1]，两者的取值范围不对称，因此需要进一步进行数学变换。这里，我们进行对数变换。这样，中立词汇值会在0附近，而反面词汇的值是负值，正向词汇的值是正值。

In [12]:
#将pos_neg_ratio的值变换成对数值
for word, ratio in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(ratio)

In [13]:
# 再次查看the、great、bad的取值
print("pos_neg_ratio for 'the' = {}".format(pos_neg_ratios['the']))
print("pos_neg_ratio for 'great' = {}".format(pos_neg_ratios['great']))
print("pos_neg_ratio for 'bad' = {}".format(pos_neg_ratios['bad']))

pos_neg_ratio for 'the' = 0.05902269426102881
pos_neg_ratio for 'great' = 0.8881047090146459
pos_neg_ratio for 'bad' = -1.3562189073456823


In [14]:
# 查看‘POSITIVE’评论中，最常出现的词汇
pos_neg_ratios.most_common()

[('edie', 4.6913478822291435),
 ('paulie', 4.0775374439057197),
 ('felix', 3.1527360223636558),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.8067217286092401),
 ('victoria', 2.6810215287142909),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.5389738710582761),
 ('flawless', 2.451005098112319),
 ('superbly', 2.2600254785752498),
 ('perfection', 2.1594842493533721),
 ('astaire', 2.1400661634962708),
 ('captures', 2.0386195471595809),
 ('voight', 2.0301704926730531),
 ('wonderfully', 2.0218960560332353),
 ('powell', 1.9783454248084671),
 ('brosnan', 1.9547990964725592),
 ('lily', 1.9203768470501485),
 ('bakshi', 1.9029851043382795),
 ('lincoln', 1.9014583864844796),
 ('refreshing', 1.8551812956655511),
 ('breathtaking', 1.8481124057791867),
 ('bourne', 1.8478489358790986),
 ('lemmon', 1.8458266904983307),
 ('delightful', 1.8002701588959635),
 ('flynn', 1.7996646487351682),
 ('andrews', 1.7764919970972666),
 ('homer', 1.7692866133759964),
 ('beautifully', 1.7626953362841438),
 ('socc

In [15]:
# 查看‘NEGATIVE’评论中，最常出现的词汇
list(reversed(pos_neg_ratios.most_common()))

[('boll', -4.9698132995760007),
 ('uwe', -4.6249728132842707),
 ('seagal', -3.6441435602725449),
 ('unwatchable', -3.2580965380214821),
 ('stinker', -3.2088254890146994),
 ('mst', -2.9502698994772336),
 ('incoherent', -2.9368917735310576),
 ('unfunny', -2.6922395950755678),
 ('waste', -2.6193845640165536),
 ('blah', -2.5704288232261625),
 ('horrid', -2.4849066497880004),
 ('pointless', -2.4553061800117097),
 ('atrocious', -2.4259083090260445),
 ('redeeming', -2.3682390632154826),
 ('prom', -2.3608540011180215),
 ('drivel', -2.3470368555648795),
 ('lousy', -2.3075726345050849),
 ('worst', -2.2869878961803778),
 ('laughable', -2.2643638801738479),
 ('awful', -2.2271942470274348),
 ('poorly', -2.2207550747464135),
 ('wasting', -2.2046046846338418),
 ('remotely', -2.1972245773362196),
 ('existent', -2.0794415416798357),
 ('boredom', -1.9951003932460849),
 ('miserably', -1.9924301646902063),
 ('sucks', -1.9870682215488209),
 ('uninspired', -1.9832976811269336),
 ('lame', -1.981767458946166)

## 3. 生成训练数据集
#### 1） 由于神经网络的输入是矩阵，而我们的原始数据是英文文本，所以，文本数据并不能直接作为模型的输入数据，而是需要进行数据转换。
#### 2） 在进行文本数据分析时，我们需要首先构建一个集合，这个集合保存了所有的文本词汇。然后，对于每一条评论中的每个词汇，可以在字典中设置1或者出现的词频。
#### 3） 对于标签数据，正面评论POSITIVE设置为1，而反面评论NEGATIVE设置为0

In [16]:
# 创建词汇集
vocab = set(total_count.keys())

In [17]:
# 词汇集中词汇的数量
vocab_size = len(vocab)
print(vocab_size)

74074


In [18]:
# 创建一个numpy array，并初始化为0，这个向量用于表示整个词汇集（词袋），每个位置代表词汇集中的一个词，而每个词具体在哪个位置并不重要。
# 但是需要确保至始至终，每个词汇所在的位置不变。
# 同时，layer_0也代表每一条输入数据
layer_0 = np.zeros((1, vocab_size))
layer_0

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [19]:
layer_0.shape

(1, 74074)

字典**word2index**，其中字典的键表示单词，字典的值表示每个单词的索引值

In [20]:
# 创建一个字典，其中字典的键表示单词，字典的值表示每个单词的索引值
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
word2index

{'': 0,
 'transpires': 1,
 'giza': 2,
 'yeah': 4,
 'gershon': 5,
 'ttm': 6,
 'randall': 7,
 'councilor': 8,
 'guinness': 36952,
 'silvio': 13,
 'procrastination': 16,
 'juncos': 17,
 'lionsault': 3,
 'celozzi': 18,
 'tftc': 19,
 'unintelligible': 21,
 'rebbe': 22,
 'foreheads': 23,
 'underling': 24,
 'perennial': 9,
 'cryptologist': 26,
 'impossibility': 27,
 'tailer': 29,
 'arcadia': 30,
 'scavenger': 10,
 'choisy': 53430,
 'husks': 32,
 'close': 70274,
 'fightfest': 34,
 'forays': 11,
 'slurs': 36,
 'carne': 37,
 'gelled': 38,
 'mobocracy': 18704,
 'melin': 42,
 'treks': 41,
 'perfidy': 12,
 'incinerate': 43,
 'plaggy': 31314,
 'vadim': 46,
 'matras': 47,
 'pasion': 48,
 'tourist': 57237,
 'improv': 12353,
 'nick': 49,
 'margareta': 50,
 'reflect': 51,
 'calvet': 53,
 'undersized': 24545,
 'magnific': 54,
 'buffalos': 61589,
 'npr': 14,
 'environs': 56,
 'performace': 49300,
 'aus': 57,
 'ilu': 61590,
 'ilsa': 60,
 'speeder': 61591,
 'hardison': 61,
 'tros': 62,
 'interconnectivity':

函数**update_input_layer**: 对每一条评论中的每个词进行迭代，对layer_0中每个位置所对应的单词在评论中每出现一次，则该位置的值加1

In [21]:
def update_input_layer(review):
    # 将layer_0声明为全局变量
    global layer_0
    # 对layer_0清零
    layer_0 *= 0
    # 迭代遍历review中的所有词，每出现一个词，对layer_0相应位置的值加1
    for word in review.split(" "):
        layer_0[0][word2index[word]] += 1

In [22]:
# 验证一下输出
update_input_layer(reviews[3])
layer_0

array([[ 167.,    0.,    0., ...,    0.,    0.,    0.]])

函数**get_target_for_label**将每一条评论的标签转换为数字0或1：0表示NEGATIVE,1表示POSITIVE

In [23]:
def get_target_for_label(label):
    if(label == 'POSITIVE'):
        return 1
    else:
        return 0

In [24]:
print('labels[0]: ', labels[0])
print(get_target_for_label(labels[0]))

labels[0]:  POSITIVE
1


## 4. 构建神经网络

### 为了方便后面的使用，我们将构建的神经网络封装在一个类中
* 在本项目中，我没有使用任何第三方深度学习框架，而是从零完整实现了一个三层神经网络模型，并前向传播得到输出值，并通过BP反向传播算法更新各层的权值，并用梯度下降方法来获得最优模型。

In [25]:
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 3, learning_rate = 0.1):
        # 首先设置一个随机数种子
        np.random.seed(1)
        # 然后对输入的reviews和labels进行数据预处理
        self.preprocess_data(reviews, labels)
        # 创建网络，包括输入数据，隐藏层神经元数量，输出节点数以及学习率
        self.init_network(len(self.review_vocab), hidden_nodes, 1, learning_rate)
        
    # 数据预处理
    def preprocess_data(self, reviews, labels):
        # 数据预处理的过程和步骤3：生成训练数据集一样
        # 创建一个集合review_vocab用于存储所有评论中出现的单词
        review_vocab = set()
        # 遍历整个评论集合，再遍历每一条评论中的单词，将单词加到review_vocab中
        for review in reviews:
            for word in review.split(" "):
                review_vocab.add(word)
        # 将review_vocab转成列表
        self.review_vocab = list(review_vocab)
        # 同理，处理标签数据集
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        self.label_vocab = list(label_vocab)
        
        # 保存词汇集和标签集的大小
        self.review_vocab_size = len(review_vocab)
        self.label_vocab_size = len(label_vocab)
        
        # 创建一个字典，其中字典的键表示单词，字典的值表示每个单词的索引值
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        # 同理，创建一个字典用于存储标签和标签索引值
        self.label2index ={}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    # 模型初始化
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # 设置号输入节点数、隐藏节点数和输出节点数以及学习率
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate
        
        # 初始化权值，这是输入层和隐藏层之间的权重
        # 输入层和隐藏层之间的权值数量是input_nodes * hidden_nodes
        # 将输入层和隐藏层之间的权重初始化为0
        self.weight_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
        
        # 隐藏层和输出层之间的权重
        # 隐藏层和输出层之间的权值数量是 hidden_nodes * output_nodes
        # 初始化的权重均值为0， 方差为output_nodes**-0.5
        self.weight_1_2 = np.random.normal(0.0, self.output_nodes ** -0.5, (self.hidden_nodes, self.output_nodes))
        # 初始化隐藏层数据为全0,该数组的大小为（1，hidden_nodes）
        self.layer_0 = np.zeros((1, input_nodes))
    
    # 输入数据更新，和步骤3中的update_input_layer函数一样
    def update_input_layer(self, review):
        # 将layer_0的数据初始化为0
        self.layer_0 *= 0
        
        # 遍历review中的每个单词，并将每个单词每出现一次，layer_0对应位置的值加1
        for word in review.split(" "):
            if(word in self.word2index.keys()):
                self.layer_0[0][self.word2index[word]] = 1
    
    # 标签数据转化为0或1
    def get_target_for_label(self, label):
        if(label == "POSITIVE"):
            return 1
        else:
            return 0
    
    # sigmoid函数
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # 定义sigmoid函数的导函数
    def sigmoid_output_2_derivative(self, output):
        return output * (1- output)
    
    
    # 训练数据
    # 在数据预处理时，对所有的评论进行了遍历，生成了词汇集（词袋），但没有生成训练数据集，因此在训练时，需要重新生成训练数据集
    # preprocess_data的输入是reviews，在train函数中，输入的参数改为training_reviews_raw和training_label
    def train(self, training_reviews, training_labels):
        assert (len(training_reviews) == len(training_labels))
        
        # 声明一个变量，记录模型的正确率，初始化正确率为0
        correct_so_far = 0
        
        # 记录开始训练开始的时间
        start = time.time()
        
        # 遍历训练数据集training_reviews和training_labels，通过对训练数据的遍历不断更新模型中的权重
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            label = training_labels[i]
            
            ## 实现前向传播算法
            
            # 输入层
            self.update_input_layer(review)
            
            # 隐藏层：用输入层的数据乘以输入层与隐藏层之间的权重
            layer_1 = self.layer_0.dot(self.weight_0_1)
            
            # 输出层：使用隐藏层的值点乘隐藏层与输出层之间的权重，并经过sigmoid函数
            layer_2 = self.sigmoid(layer_1.dot(self.weight_1_2))
            
            
            ## 实现反向传播算法
            # 计算输出的误差：输出数值和真实数值之间的差值
            layer_2_error = layer_2 - self.get_target_for_label(label)
            # 输出层的权值调整量
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2) 
            
            # 反向传播到隐藏层的误差
            layer_1_error = layer_2_delta.dot(self.weight_1_2.T) # 误差传播的隐藏层
            layer_1_delta = layer_1_error # 隐藏层的梯度- 不是非线性变换，所以与误差值相同
            
            # 更新权值
            # 使用输出层的梯度值来更新隐藏层到输出层之间的权重
            self.weight_1_2 -= layer_1.T.dot(layer_2_delta) * self.learning_rate
            # 使用隐藏层的梯度值来更新输入层到隐藏层之间的权重
            self.weight_0_1 -= self.layer_0.T.dot(layer_1_delta) * self.learning_rate
        
            # 跟踪模型训练的准确率
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # 为了调试的目的，打印训练过程中的预测准确率和训练速度
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if (i % 2500 == 0):
                print("")
    def test(self, testing_reviews, testing_labels):
        # 跟踪训练数据的预测准确率
        correct = 0
        
        # 记录测试开始的时间，用于计算测试集运行速度
        start = time.time()
        
        # 遍历测试数据集中的每一条评论，并进行预测
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if (pred == testing_labels[i]):
                correct += 1
                
            # 打印测试信息，与训练过程相同
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
        
    def run(self, review):
        # 输入层
        self.update_input_layer(review.lower())
        # 隐藏层
        layer_1 = self.layer_0.dot(self.weight_0_1)
        # 输出层
        layer_2 = self.sigmoid(layer_1.dot(self.weight_1_2))
        # 判断输出属性
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"

In [26]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):256.0 #Correct:1946 #Trained:2501 Training Accuracy:77.8%
Progress:20.8% Speed(reviews/sec):265.2 #Correct:4000 #Trained:5001 Training Accuracy:79.9%
Progress:31.2% Speed(reviews/sec):265.1 #Correct:6123 #Trained:7501 Training Accuracy:81.6%
Progress:41.6% Speed(reviews/sec):268.6 #Correct:8282 #Trained:10001 Training Accuracy:82.8%
Progress:52.0% Speed(reviews/sec):261.1 #Correct:10436 #Trained:12501 Training Accuracy:83.4%
Progress:62.5% Speed(reviews/sec):250.4 #Correct:12574 #Trained:15001 Training Accuracy:83.8%
Progress:72.9% Speed(reviews/sec):252.1 #Correct:14691 #Trained:17501 Training Accuracy:83.9%
Progress:83.3% Speed(reviews/sec):254.6 #Correct:16861 #Trained:20001 Training Accuracy:84.3%
Progress:93.7% Speed(reviews/sec):255.6 #Correct:19048 #Trained:22501 Training Accuracy:84.6%
Progress:99.9% Speed(reviews/sec):255.0 #Correct:20369 #Trained:24000 Training

In [27]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1023. #Correct:859 #Tested:1000 Testing Accuracy:85.9%

***从上面的模型，主要存在两个问题，分别是模型复杂度和训练速度***

**1.模型复杂度**
* 对上面的模型，我分别设定了隐藏单元的数量为3和10，发现两次训练的结果的准确率一样。
* 由此可见，在设定隐藏层单元数量为3的时候，模型的复杂度已经足够好。增加隐藏层单元的数量即增大模型的复杂度并不会使得模型的准确率增加，随之带来的负面影响是训练速度的下降。
* 神经网络具有很强的学习能力，但强学习能力是建立在大数据集的基础上。如果数据量不大，模型复杂度很大，会使得模型欠拟合。

** 2. 训练速度**
* 从上面的数据可以看出，模型训练的速度很慢，一秒钟仅能处理不到100条评论，问题主要出现这矩阵相乘上，大量的矩阵相乘在CPU环境下，速度很慢。
* 下面通过改进模型，提高模型的训练速递。

## 5. 神经网络模型改进：提升训练速度

在前一个模型中，已经发现，模型训练的速度很慢，效率很低，主要原因在于，输入数据是（1， 74074）的数据，数据维度大，在进行前向传播和反向传播的时候，矩阵相乘造成很大的计算资源开销。进一步分析，发现，其实一条评论的长度也就是几十个最多几百个单词。但是词袋中的单词量有7万多，这样数据数据其实是稀疏矩阵，存在大量的0，但是这些0对模型的训练并不起作用，却要大量地参与计算。
所以，本节通过让输入数据中0项不要参与整个模型的计算：前向和反向权值更新来提供训练速度，从而提升训练效率。
* 在新的模型中，我们不使用update_input_layer函数来处理输入数据。取而代之的是使用一个列表来记录词汇在词袋中出现的索引值。

In [32]:
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 3, learning_rate = 0.1):
        # 首先设置一个随机数种子
        np.random.seed(1)
        # 然后对输入的reviews和labels进行数据预处理
        self.preprocess_data(reviews, labels)
        # 创建网络，包括输入数据，隐藏层神经元数量，输出节点数以及学习率
        self.init_network(len(self.review_vocab), hidden_nodes, 1, learning_rate)
        
    # 数据预处理
    def preprocess_data(self, reviews, labels):
        # 数据预处理的过程和步骤3：生成训练数据集一样
        # 创建一个集合review_vocab用于存储所有评论中出现的单词
        review_vocab = set()
        # 遍历整个评论集合，再遍历每一条评论中的单词，将单词加到review_vocab中
        for review in reviews:
            for word in review.split(" "):
                review_vocab.add(word)
        # 将review_vocab转成列表
        self.review_vocab = list(review_vocab)
        # 同理，处理标签数据集
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        self.label_vocab = list(label_vocab)
        
        # 保存词汇集和标签集的大小
        self.review_vocab_size = len(review_vocab)
        self.label_vocab_size = len(label_vocab)
        
        # 创建一个字典，其中字典的键表示单词，字典的值表示每个单词的索引值
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        # 同理，创建一个字典用于存储标签和标签索引值
        self.label2index ={}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    # 模型初始化
    # 在这个函数中
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # 设置号输入节点数、隐藏节点数和输出节点数以及学习率
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate
        
        # 初始化权值，这是输入层和隐藏层之间的权重
        # 输入层和隐藏层之间的权值数量是input_nodes * hidden_nodes
        # 将输入层和隐藏层之间的权重初始化为0
        self.weight_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
        
        # 隐藏层和输出层之间的权重
        # 隐藏层和输出层之间的权值数量是 hidden_nodes * output_nodes
        # 初始化的权重均值为0， 方差为output_nodes**-0.5
        self.weight_1_2 = np.random.normal(0.0, self.output_nodes ** -0.5, (self.hidden_nodes, self.output_nodes))
        # 此处我们不再需要输入数据层layer_0这个变量
        #定义隐藏层单元并初始化
        self.layer_1 = np.zeros((1, hidden_nodes))
        
    # 删除update_input_layer函数
    
    # 标签数据转化为0或1
    def get_target_for_label(self, label):
        if(label == "POSITIVE"):
            return 1
        else:
            return 0
    
    # sigmoid函数
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # 定义sigmoid函数的导函数
    def sigmoid_output_2_derivative(self, output):
        return output * (1- output)
    
    
    # 训练数据
    # 在数据预处理时，对所有的评论进行了遍历，生成了词汇集（词袋），但没有生成训练数据集，因此在训练时，需要重新生成训练数据集
    # preprocess_data的输入是reviews，在train函数中，输入的参数改为training_reviews_raw和training_label
    def train(self, training_reviews_raws, training_labels):
        # training_reviews是一个列表，用于保存训练数据
        training_reviews = list()
        # 遍历每一条评论中的每个词，如果该词出现在词袋中，则根据word2index获取该词对应的索引号，并保存索引号
        # 因为只有在评论中出现的词才参与模型中权值的更新，因此我们只需要保存这些词的索引号即可
        # 将索引号列表加到training_review中
        for review in training_reviews_raws:
            # 只让内容不为0的项目参与运算，因此indices保存了输入数据中不为0的索引。
            indices = set()
            for word in review.split(" "):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))
        assert (len(training_reviews) == len(training_labels))
        
        # 声明一个变量，记录模型的正确率，初始化正确率为0
        correct_so_far = 0
        
        # 记录开始训练开始的时间
        start = time.time()
        
        # 遍历训练数据集training_reviews和training_labels，通过对训练数据的遍历不断更新模型中的权重
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            # 此时的review是一个列表，里面保存的是各个单词在word2index中的值，也即单词的索引值
            label = training_labels[i]
            
            ## 实现前向传播算法
            
            # 隐藏层：用输入层的数据乘以输入层与隐藏层之间的权重
            self.layer_1 *= 0
            for index in review:
                # 此时不再使用矩阵相乘，而是对输入数据的非零项对应的权值乘以1累加到layer_1中
                self.layer_1 += (self.weight_0_1[index] * 1) # 1代表输入数据中对应索引位置出现了该单词
            
            # 输出层：使用隐藏层的值点乘隐藏层与输出层之间的权重，并经过sigmoid函数
            layer_2 = self.sigmoid(self.layer_1.dot(self.weight_1_2))
            
            
            ## 实现反向传播算法
            # 计算输出的误差：输出数值和真实数值之间的差值
            layer_2_error = layer_2 - self.get_target_for_label(label)
            # 输出层的权值调整量
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2) 
            
            # 反向传播到隐藏层的误差
            layer_1_error = layer_2_delta.dot(self.weight_1_2.T) # 误差传播的隐藏层
            layer_1_delta = layer_1_error # 隐藏层的梯度- 不是非线性变换，所以与误差值相同
            
            # 更新权值
            # 使用输出层的梯度值来更新隐藏层到输出层之间的权重
            self.weight_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate
            # 更新正向传播从输入层到隐藏层的权重
            for index in review:
                self.weight_0_1[index] -= layer_1_delta[0] * self.learning_rate
        
            # 跟踪模型训练的准确率
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # 为了调试的目的，打印训练过程中的预测准确率和训练速度
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if (i % 2500 == 0):
                print("")
    def test(self, testing_reviews, testing_labels):
        # 跟踪训练数据的预测准确率
        correct = 0
        
        # 记录测试开始的时间，用于计算测试集运行速度
        start = time.time()
        
        # 遍历测试数据集中的每一条评论，并进行预测
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if (pred == testing_labels[i]):
                correct += 1
                
            # 打印测试信息，与训练过程相同
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
        
    def run(self, review):
        
        # 隐藏层
        self.layer_1 *= 0
        unique_indices = set()
        for word in review.lower().split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += (self.weight_0_1[index] * 1)
        
        # 输出层
        layer_2 = self.sigmoid(self.layer_1.dot(self.weight_1_2))
        # 判断输出属性
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"

In [33]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):950.1 #Correct:1946 #Trained:2501 Training Accuracy:77.8%
Progress:20.8% Speed(reviews/sec):927.9 #Correct:4000 #Trained:5001 Training Accuracy:79.9%
Progress:31.2% Speed(reviews/sec):958.0 #Correct:6123 #Trained:7501 Training Accuracy:81.6%
Progress:41.6% Speed(reviews/sec):933.9 #Correct:8282 #Trained:10001 Training Accuracy:82.8%
Progress:52.0% Speed(reviews/sec):942.0 #Correct:10436 #Trained:12501 Training Accuracy:83.4%
Progress:62.5% Speed(reviews/sec):862.9 #Correct:12574 #Trained:15001 Training Accuracy:83.8%
Progress:72.9% Speed(reviews/sec):789.8 #Correct:14691 #Trained:17501 Training Accuracy:83.9%
Progress:83.3% Speed(reviews/sec):810.5 #Correct:16861 #Trained:20001 Training Accuracy:84.3%
Progress:93.7% Speed(reviews/sec):822.8 #Correct:19048 #Trained:22501 Training Accuracy:84.6%
Progress:99.9% Speed(reviews/sec):821.7 #Correct:20369 #Trained:24000 Training

In [34]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1172. #Correct:859 #Tested:1000 Testing Accuracy:85.9%

* 比较两个模型，可以看到，改进的第二个模型的运行速度提升幅度很大。
* 两个模型的原理完全相同，只是在实现权值更新的数学计算中有效利用了矩阵相乘的数学性质，从而加快了运算速度。
* 下一步的目标是提升模型的准确率。

## 6. 神经网络模型改进：提升模型准确率
* 解决了模型运行速度的问题，而模型的准确率也达到了85%左右。下面，通过对数据集的进一步预处理，去除噪声，验证模型准确率的提升情况。
* 在数据探索阶段，通过计算正面词汇和反面词汇的比例，可以看到有很多出现频次很高，但是不具有情感倾向的中立词，我们可以尝试去除这些中立词，使得输入数据的感情色彩更加鲜明，然后观察模型的准确率变化情况。
* 需要指出的是，由于在本模型中，输入的数据是只包含0或1的向量，也就是某个单词是否出现，而不是词频，因此其实去除中立词对模型的准确率的提升不会很大。而如果输入数据是词频数据，那么去除出现频率高的中立词对模型准确率的提升应该会比较明显。不过，还是需要进行试验验证。

In [42]:
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 3, learning_rate = 0.1, min_count = 10, polarity_cutoff = 0.1):
        # 首先设置一个随机数种子
        np.random.seed(1)
        # 然后对输入的reviews和labels进行数据预处理
        self.preprocess_data(reviews, labels, min_count, polarity_cutoff)
        # 创建网络，包括输入数据，隐藏层神经元数量，输出节点数以及学习率
        self.init_network(len(self.review_vocab), hidden_nodes, 1, learning_rate)
        
    # 数据预处理
    def preprocess_data(self, reviews, labels, min_count, polarity_cutoff):
        # 定义三个Counter对象，用于存储正向评论和反向评论中出现的词频
        positive_count = Counter()
        negative_count = Counter()
        total_count = Counter()
        
        for i in range(len(reviews)):
            if (labels[i] == 'POSITIVE'):
                for word in reviews[i].split(" "):
                    positive_count[word] += 1
                    total_count[word] += 1
            else:
                for word in reviews[i].split(" "):
                    negative_count[word] += 1
                    total_count[word] += 1
        pos_neg_ratios = Counter()
        
        for word, cnt in total_count.most_common():
            if(cnt >= 50):
                pos_neg_ratios[word] = positive_count[word] / float(negative_count[word] + 1)
                pos_neg_ratios[word] = pos_neg_ratio
        for word, ratio in pos_neg_ratios.most_common():
            if (ratio > 1):
                pos_neg_ratios[word] = np.log(ratio)
            else:
                pos_neg_ratios[word] = -np.log((1 / (ratio + 0.01)))         
                
        # 数据预处理的过程和步骤3：生成训练数据集一样
        # 创建一个集合review_vocab用于存储所有评论中出现的单词
        review_vocab = set()
        # 遍历整个评论集合，再遍历每一条评论中的单词，将单词加到review_vocab中
        for review in reviews:
            for word in review.split(" "):
                # 增加判断条件，对满足最小词频数或者最小正向反向词频比率的单词才加到review_cab中
                if (word in pos_neg_ratios.keys()):
                    if((pos_neg_ratios[word] >= polarity_cutoff) or (pos_neg_ratios[word] <= -polarity_cutoff)):
                        review_vocab.add(word)
        # 将review_vocab转成列表
        self.review_vocab = list(review_vocab)
        # 同理，处理标签数据集
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        self.label_vocab = list(label_vocab)
        
        # 保存词汇集和标签集的大小
        self.review_vocab_size = len(review_vocab)
        self.label_vocab_size = len(label_vocab)
        
        # 创建一个字典，其中字典的键表示单词，字典的值表示每个单词的索引值
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        # 同理，创建一个字典用于存储标签和标签索引值
        self.label2index ={}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    # 模型初始化
    # 在这个函数中
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # 设置号输入节点数、隐藏节点数和输出节点数以及学习率
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate
        
        # 初始化权值，这是输入层和隐藏层之间的权重
        # 输入层和隐藏层之间的权值数量是input_nodes * hidden_nodes
        # 将输入层和隐藏层之间的权重初始化为0
        self.weight_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
        
        # 隐藏层和输出层之间的权重
        # 隐藏层和输出层之间的权值数量是 hidden_nodes * output_nodes
        # 初始化的权重均值为0， 方差为output_nodes**-0.5
        self.weight_1_2 = np.random.normal(0.0, self.output_nodes ** -0.5, (self.hidden_nodes, self.output_nodes))
        # 此处我们不再需要输入数据层layer_0这个变量
        #定义隐藏层单元并初始化
        self.layer_1 = np.zeros((1, hidden_nodes))
        
    # 删除update_input_layer函数
    
    # 标签数据转化为0或1
    def get_target_for_label(self, label):
        if(label == "POSITIVE"):
            return 1
        else:
            return 0
    
    # sigmoid函数
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # 定义sigmoid函数的导函数
    def sigmoid_output_2_derivative(self, output):
        return output * (1- output)
    
    
    # 训练数据
    # 在数据预处理时，对所有的评论进行了遍历，生成了词汇集（词袋），但没有生成训练数据集，因此在训练时，需要重新生成训练数据集
    # preprocess_data的输入是reviews，在train函数中，输入的参数改为training_reviews_raw和training_label
    def train(self, training_reviews_raws, training_labels):
        # training_reviews是一个列表，用于保存训练数据
        training_reviews = list()
        # 遍历每一条评论中的每个词，如果该词出现在词袋中，则根据word2index获取该词对应的索引号，并保存索引号
        # 因为只有在评论中出现的词才参与模型中权值的更新，因此我们只需要保存这些词的索引号即可
        # 将索引号列表加到training_review中
        for review in training_reviews_raws:
            # 只让内容不为0的项目参与运算，因此indices保存了输入数据中不为0的索引。
            indices = set()
            for word in review.split(" "):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))
            
        assert (len(training_reviews) == len(training_labels))
        
        # 声明一个变量，记录模型的正确率，初始化正确率为0
        correct_so_far = 0
        
        # 记录开始训练开始的时间
        start = time.time()
        
        # 遍历训练数据集training_reviews和training_labels，通过对训练数据的遍历不断更新模型中的权重
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            # 此时的review是一个列表，里面保存的是各个单词在word2index中的值，也即单词的索引值
            label = training_labels[i]
            
            ## 实现前向传播算法
            
            # 隐藏层：用输入层的数据乘以输入层与隐藏层之间的权重
            self.layer_1 *= 0
            for index in review:
                # 此时不再使用矩阵相乘，而是对输入数据的非零项对应的权值乘以1累加到layer_1中
                self.layer_1 += (self.weight_0_1[index] * 1) # 1代表输入数据中对应索引位置出现了该单词
            
            # 输出层：使用隐藏层的值点乘隐藏层与输出层之间的权重，并经过sigmoid函数
            layer_2 = self.sigmoid(self.layer_1.dot(self.weight_1_2))
            
            
            ## 实现反向传播算法
            # 计算输出的误差：输出数值和真实数值之间的差值
            layer_2_error = layer_2 - self.get_target_for_label(label)
            # 输出层的权值调整量
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2) 
            
            # 反向传播到隐藏层的误差
            layer_1_error = layer_2_delta.dot(self.weight_1_2.T) # 误差传播的隐藏层
            layer_1_delta = layer_1_error # 隐藏层的梯度- 不是非线性变换，所以与误差值相同
            
            # 更新权值
            # 使用输出层的梯度值来更新隐藏层到输出层之间的权重
            self.weight_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate
            # 更新正向传播从输入层到隐藏层的权重
            for index in review:
                self.weight_0_1[index] -= layer_1_delta[0] * self.learning_rate
        
            # 跟踪模型训练的准确率
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # 为了调试的目的，打印训练过程中的预测准确率和训练速度
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if (i % 2500 == 0):
                print("")
    def test(self, testing_reviews, testing_labels):
        # 跟踪训练数据的预测准确率
        correct = 0
        
        # 记录测试开始的时间，用于计算测试集运行速度
        start = time.time()
        
        # 遍历测试数据集中的每一条评论，并进行预测
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if (pred == testing_labels[i]):
                correct += 1
                
            # 打印测试信息，与训练过程相同
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
        
    def run(self, review):
        
        # 隐藏层
        self.layer_1 *= 0
        unique_indices = set()
        for word in review.lower().split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += (self.weight_0_1[index] * 1)
        
        # 输出层
        layer_2 = self.sigmoid(self.layer_1.dot(self.weight_1_2))
        # 判断输出属性
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"

In [43]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], min_count=20, polarity_cutoff=0.05, learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):879.9 #Correct:1960 #Trained:2501 Training Accuracy:78.3%
Progress:20.8% Speed(reviews/sec):1002. #Correct:4018 #Trained:5001 Training Accuracy:80.3%
Progress:31.2% Speed(reviews/sec):1022. #Correct:6131 #Trained:7501 Training Accuracy:81.7%
Progress:41.6% Speed(reviews/sec):998.7 #Correct:8287 #Trained:10001 Training Accuracy:82.8%
Progress:52.0% Speed(reviews/sec):1029. #Correct:10442 #Trained:12501 Training Accuracy:83.5%
Progress:62.5% Speed(reviews/sec):1015. #Correct:12571 #Trained:15001 Training Accuracy:83.8%
Progress:72.9% Speed(reviews/sec):1034. #Correct:14695 #Trained:17501 Training Accuracy:83.9%
Progress:83.3% Speed(reviews/sec):1049. #Correct:16885 #Trained:20001 Training Accuracy:84.4%
Progress:93.7% Speed(reviews/sec):1037. #Correct:19067 #Trained:22501 Training Accuracy:84.7%
Progress:99.9% Speed(reviews/sec):1045. #Correct:20385 #Trained:24000 Training

In [44]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):773.1 #Correct:857 #Tested:1000 Testing Accuracy:85.7%

* 正如分析，模型的准确率提升并不大，主要原因是我们使用的输入数据不是词频数据，而是某个词汇是否出现在评论中。
* 由于减少了词袋中的词汇数量，因此在模型训练过程中，训练速度也有明显提升。
* 对于使用词频作为输入数据，去掉停止词和中立词会对模型的准确率有很大的影响。

## 7. 项目总结


* 本项目通过使用神经网络构建了文本情感分类器模型，模型的分类准确率在85%左右。
* 神经网络具有很强的学习能力，但是相比传统机器学习算法，由于神经网络的隐藏层可以有很多神经元，隐藏层的层数可以很多，因此神经网络模型的训练速度不会很快。这就需要借助算法优化以及GPU等高性能计算硬件来提升训练速度。
* 文本分类的一般步骤通常需要首先构建一个词袋模型，通过词袋来表示统一格式的输入数据。输入数据可以是只含有0或1的向量，也可以是包含文本词频的向量，在一些文章中指出，两种表示方法的模型准确率差不多。
* 在算法方面，可以进行文本分类的模型很多，主要包括传统机器学习的朴素贝叶斯方法、神经网络以及深度学习的卷积神经网络等。